In [42]:
import praw
import pandas as pd
import datetime

import xlsxwriter

import requests
import urllib
from tqdm import tqdm
import time
import os

In [43]:
# # SETTINGS
# download_pictures = False
# meme_number = 1000 # limited to 1000 by PRAW -> use pushshift API?

In [44]:
# def get_date(submission):
#     time = submission.created
#     return datetime.datetime.fromtimestamp(time)

In [45]:
# def save_image(url):
    
#     file_name = url.split("/")[-1]
#     res = requests.get(url, stream = True)

#     # Creating an images folder if it doens't already exist
#     if not os.path.exists('images'):
#        os.makedirs('images')

#     urllib.request.urlretrieve(url, "images/" + file_name)

In [46]:
# reddit_read_only = praw.Reddit(
#     client_id='JzbofoB1-Xrh3gbv--LVTQ',
#     client_secret='v3R81WmJnSJJ8WWdjn4sZvRcerIVGg',
#     user_agent='Ukraine Scraper'
# )
# # qwerrtty423499 username
# # KVKb6AqeabAG2PD password
# # cveodyiphyxqccujnc@tmmbt.com email
# # v3R81WmJnSJJ8WWdjn4sZvRcerIVGg secret

# subreddit = reddit_read_only.subreddit("memes")

# # posts = subreddit.hot(limit=10)
# posts = subreddit.top(time_filter='year', limit=meme_number)
# # Scraping the top posts of the current month



# posts_dict = {"Title": [], "Author" : [], "Timestamp" : [], "Post Text": [],
#             "ID": [], "Score": [],
#             "Total Comments": [], "Image URL": []
#             }
# # t = tqdm(total=meme_number, bar_format='{l_bar}{bar:50}{r_bar}{bar:-1b}')
# t = tqdm(total=meme_number, bar_format='{l_bar}{bar:50}{r_bar}{bar:-1b}')
# with t as progress_bar:
#     for post in posts:
#         # URL of each post
#         # ensures posts without images are ignored
#         if (".jpg" in post.url or ".png" in post.url or ".tiff" in post.url or ".gif" in post.url):
#             # image is valid
#             posts_dict["Image URL"].append(str(post.url))
#             if (download_pictures):
#                 download_pictures(str(post.url))

#         else:
#             progress_bar.update()
#             continue 

#         # Title of each post
#         posts_dict["Title"].append(post.title)

#         # Author of each post
#         posts_dict["Author"].append(str(post.author))

#         # Time post was created
#         posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post.created).strftime("%m/%d/%Y, %H:%M:%S"))

#         # Text inside a post
#         posts_dict["Post Text"].append(post.selftext)

#         # Unique ID of each post
#         posts_dict["ID"].append(post.id)

#         # The score of a post
#         posts_dict["Score"].append(post.score)

#         # Total number of comments inside the post
#         posts_dict["Total Comments"].append(post.num_comments)
        
#         progress_bar.update()


# # Displaying the data in a pandas dataframe (for testing purposes)
# # pd.set_option('display.max_rows', 100)
# # pd.set_option('max_colwidth', None)

# # top_posts = pd.DataFrame(posts_dict)
# # top_posts



In [47]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------
def write_data(posts_dict):
    workbook = xlsxwriter.Workbook('ukrainememes_subreddit.xlsx')
    worksheet = workbook.add_worksheet()

    # column, row

    # Writing the table headers
    i = 0
    for key in posts_dict:
        worksheet.write(0, i, key)
        i+=1

    # Writing the table data
    for x in range(1, len(posts_dict["Title"])):
        i = 0
        for key in posts_dict:
            if (key == "Image URL"):
                worksheet.write_url(x, i, posts_dict[key][x])
            else:
                worksheet.write(x, i, posts_dict[key][x])
            i+=1

    # Saving the document
    workbook.close()


In [48]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------
def read_posts(post_list, posts_dict):
    for post in post_list:
        
        if (".jpg" in post["url"] or ".png" in post["url"] or ".tiff" in post["url"] or ".gif" in post["url"]):
            posts_dict["Image URL"].append(str(post["url"]))
#             if (download_pictures):
#                 download_pictures(str(post["url"]))
        else:
            continue 

        # Title of each post
        posts_dict["Title"].append(post["title"])

        # Author of each post
        posts_dict["Author"].append(str(post["author"]))

        # Time post was created
        posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post["created_utc"]).strftime("%m/%d/%Y, %H:%M:%S"))

        # Text inside a post
        posts_dict["Post Text"].append(post["selftext"])

        # Unique ID of each post
        posts_dict["ID"].append(post["id"])

        # The score of a post
        posts_dict["Score"].append(post["score"])

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(str(post["num_comments"]))
        
        
    

In [49]:
# ------- MEMES FROM THE "ukrainememes" SUBREDDIT -------

from pmaw import PushshiftAPI
# http://web.archive.org/web/20210123052853/https://pushshift.io/api-parameters/
# https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/

api = PushshiftAPI()
print(time.time())

# Settings
subreddit = "ukrainememes"
chunk_size = 100
reps = 3

posts_dict = {
    "Title": [], "Author" : [], "Timestamp" : [], "Post Text": [], 
    "ID": [], "Score": [], "Total Comments": [], "Image URL": []
}
    
# Iterating over the returned list of submissions
for x in range(reps):
    if (x == 0):
        last_time = int(time.time())
    else:
        if (len(post_list) != 0):
            last_time = post_list[-1]["created_utc"]
        else:
            break
    
    posts = api.search_submissions(
        subreddit=subreddit, 
        limit=chunk_size,
        before=last_time,
        sort="desc",
        sort_type="created_utc"
    )
    post_list = [post for post in posts]
    read_posts(post_list, posts_dict)

# Displaying the data in a pandas dataframe (for testing purposes)
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', None)

top_posts = pd.DataFrame(posts_dict)
top_posts

# Writing the data to excel
write_data(posts_dict)



    

1673222280.3338306
